In [1]:
%cd /Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


#### First run in terminal:
%docker run --rm -p 9200:9200 -p 9300:9300 -e "xpack.security.enabled=false" -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:8.7.0

In [2]:
import sys
import os
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)

from read_data.read_data import read_urls_questions, get_questions, get_url_content_tuples, get_relevant_docs
from elasticsearch import Elasticsearch
from retrieval.sparse_retrieval.bm25 import set_index, get_result_tuples

In [3]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:
#es_client.info().body

In [6]:
collected, questions =  read_urls_questions(os.path.join('data/reference_urls/reference_urls_collected.csv'),
                                            os.path.join('data/question_answer/questions_updated_urls.csv'),
                                            clean_url_nan=True) # read collected urls and questions + remove unsuccessful collection

In [7]:
question_list = get_questions(questions)
document_list = get_url_content_tuples(collected)
mappings = {
        "properties": {
            "url": {"type": "text"},
            "text": {"type": "text", "analyzer": "standard", "similarity": "BM25"}
    }
}

In [8]:
es_client = set_index(es_client=es_client, collected=collected, mappings=mappings)


In [9]:
results = get_result_tuples(es_client=es_client, questions=question_list)

In [10]:
from retrieval.evaluation.evaluate import calculate_recall_at_k, calculate_average_recall

In [11]:
relevant_docs = get_relevant_docs(questions)
recalls = calculate_recall_at_k(question_list, results, relevant_docs, [1, 5, 10], 'bm25') # think of a metric similarity recall? 
calculate_average_recall(recalls)

{1: 0.08823529411764706, 5: 0.34558823529411764, 10: 0.6090686274509804}

#### Run to save results

In [12]:
import pickle
with open ('data/results/bm25_results_short.pickle', 'wb') as f:
    pickle.dump(results, f)